# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 12 2022 11:49AM,246154,10,MSP213856,"Methapharm, Inc.",Released
1,Sep 12 2022 11:49AM,246154,10,MSP213858,"Methapharm, Inc.",Released
2,Sep 12 2022 11:49AM,246154,10,MSP213859,"Methapharm, Inc.",Released
3,Sep 12 2022 11:49AM,246154,10,MSP213860,"Methapharm, Inc.",Released
4,Sep 12 2022 11:49AM,246154,10,MSP213862,"Methapharm, Inc.",Released
5,Sep 12 2022 11:49AM,246154,10,MSP213863,"Methapharm, Inc.",Released
6,Sep 12 2022 11:49AM,246154,10,MSP213866,"Methapharm, Inc.",Released
7,Sep 12 2022 11:49AM,246154,10,MSP213875,"Methapharm, Inc.",Released
8,Sep 12 2022 11:49AM,246154,10,MSP213867,"Methapharm, Inc.",Released
9,Sep 12 2022 11:49AM,246154,10,MSP213869,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,246148,Released,22
27,246149,Released,1
28,246150,Released,1
29,246152,Released,1
30,246154,Released,21


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246148,NaN,NaN,22.0
246149,NaN,NaN,1.0
246150,NaN,NaN,1.0
246152,NaN,NaN,1.0
246154,NaN,NaN,21.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246060,0.0,0.0,1.0
246096,0.0,1.0,0.0
246103,3.0,4.0,0.0
246106,0.0,0.0,1.0
246109,0.0,13.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246060,0,0,1
246096,0,1,0
246103,3,4,0
246106,0,0,1
246109,0,13,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246060,0,0,1
1,246096,0,1,0
2,246103,3,4,0
3,246106,0,0,1
4,246109,0,13,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246060,,,1
1,246096,,1,
2,246103,3,4,
3,246106,,,1
4,246109,,13,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 12 2022 11:49AM,246154,10,"Methapharm, Inc."
21,Sep 12 2022 11:46AM,246152,10,"Senseonics, Incorporated"
22,Sep 12 2022 11:42AM,246150,10,"Methapharm, Inc."
23,Sep 12 2022 11:40AM,246149,19,"Granules Pharmaceuticals, Inc."
24,Sep 12 2022 11:38AM,246148,19,"AdvaGen Pharma, Ltd"
46,Sep 12 2022 11:33AM,246147,21,"NBTY Global, Inc."
47,Sep 12 2022 11:22AM,246144,21,"NBTY Global, Inc."
48,Sep 12 2022 10:55AM,246143,10,"Citieffe, Inc."
49,Sep 12 2022 10:40AM,246142,16,"Emersa Waterbox, LLC"
50,Sep 12 2022 10:25AM,246131,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 12 2022 11:49AM,246154,10,"Methapharm, Inc.",,,21
1,Sep 12 2022 11:46AM,246152,10,"Senseonics, Incorporated",,,1
2,Sep 12 2022 11:42AM,246150,10,"Methapharm, Inc.",,,1
3,Sep 12 2022 11:40AM,246149,19,"Granules Pharmaceuticals, Inc.",,,1
4,Sep 12 2022 11:38AM,246148,19,"AdvaGen Pharma, Ltd",,,22
5,Sep 12 2022 11:33AM,246147,21,"NBTY Global, Inc.",,,1
6,Sep 12 2022 11:22AM,246144,21,"NBTY Global, Inc.",,1,
7,Sep 12 2022 10:55AM,246143,10,"Citieffe, Inc.",,,1
8,Sep 12 2022 10:40AM,246142,16,"Emersa Waterbox, LLC",,1,
9,Sep 12 2022 10:25AM,246131,20,"ACI Healthcare USA, Inc.",,11,24


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 12 2022 11:49AM,246154,10,"Methapharm, Inc.",21,,
1,Sep 12 2022 11:46AM,246152,10,"Senseonics, Incorporated",1,,
2,Sep 12 2022 11:42AM,246150,10,"Methapharm, Inc.",1,,
3,Sep 12 2022 11:40AM,246149,19,"Granules Pharmaceuticals, Inc.",1,,
4,Sep 12 2022 11:38AM,246148,19,"AdvaGen Pharma, Ltd",22,,
5,Sep 12 2022 11:33AM,246147,21,"NBTY Global, Inc.",1,,
6,Sep 12 2022 11:22AM,246144,21,"NBTY Global, Inc.",,1,
7,Sep 12 2022 10:55AM,246143,10,"Citieffe, Inc.",1,,
8,Sep 12 2022 10:40AM,246142,16,"Emersa Waterbox, LLC",,1,
9,Sep 12 2022 10:25AM,246131,20,"ACI Healthcare USA, Inc.",24,11,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 12 2022 11:49AM,246154,10,"Methapharm, Inc.",21,,
1,Sep 12 2022 11:46AM,246152,10,"Senseonics, Incorporated",1,,
2,Sep 12 2022 11:42AM,246150,10,"Methapharm, Inc.",1,,
3,Sep 12 2022 11:40AM,246149,19,"Granules Pharmaceuticals, Inc.",1,,
4,Sep 12 2022 11:38AM,246148,19,"AdvaGen Pharma, Ltd",22,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 12 2022 11:49AM,246154,10,"Methapharm, Inc.",21.0,NaN,NaN
1,Sep 12 2022 11:46AM,246152,10,"Senseonics, Incorporated",1.0,NaN,NaN
2,Sep 12 2022 11:42AM,246150,10,"Methapharm, Inc.",1.0,NaN,NaN
3,Sep 12 2022 11:40AM,246149,19,"Granules Pharmaceuticals, Inc.",1.0,NaN,NaN
4,Sep 12 2022 11:38AM,246148,19,"AdvaGen Pharma, Ltd",22.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 12 2022 11:49AM,246154,10,"Methapharm, Inc.",21.0,0.0,0.0
1,Sep 12 2022 11:46AM,246152,10,"Senseonics, Incorporated",1.0,0.0,0.0
2,Sep 12 2022 11:42AM,246150,10,"Methapharm, Inc.",1.0,0.0,0.0
3,Sep 12 2022 11:40AM,246149,19,"Granules Pharmaceuticals, Inc.",1.0,0.0,0.0
4,Sep 12 2022 11:38AM,246148,19,"AdvaGen Pharma, Ltd",22.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2707486,113.0,22.0,0.0
12,246113,1.0,1.0,0.0
15,738377,16.0,18.0,0.0
16,246142,0.0,1.0,0.0
19,738400,23.0,4.0,3.0
20,492237,25.0,11.0,0.0
21,984447,2.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2707486,113.0,22.0,0.0
1,12,246113,1.0,1.0,0.0
2,15,738377,16.0,18.0,0.0
3,16,246142,0.0,1.0,0.0
4,19,738400,23.0,4.0,3.0
5,20,492237,25.0,11.0,0.0
6,21,984447,2.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,113.0,22.0,0.0
1,12,1.0,1.0,0.0
2,15,16.0,18.0,0.0
3,16,0.0,1.0,0.0
4,19,23.0,4.0,3.0
5,20,25.0,11.0,0.0
6,21,2.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,113.0
1,12,Released,1.0
2,15,Released,16.0
3,16,Released,0.0
4,19,Released,23.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,3.0,0.0,0.0
Executing,22.0,1.0,18.0,1.0,4.0,11.0,2.0
Released,113.0,1.0,16.0,0.0,23.0,25.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,3.0,0.0,0.0
1,Executing,22.0,1.0,18.0,1.0,4.0,11.0,2.0
2,Released,113.0,1.0,16.0,0.0,23.0,25.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,3.0,0.0,0.0
1,Executing,22.0,1.0,18.0,1.0,4.0,11.0,2.0
2,Released,113.0,1.0,16.0,0.0,23.0,25.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()